In [14]:
import csv
import random
import math

# Function to read customer data from a CSV file
def read_customers_from_csv(file_path):
    customers = []
    with open(file_path, mode='r') as file:
        reader = csv.DictReader(file)
        for row in reader:
            latitude = float(row['Latitude'])
            longitude = float(row['Longitude'])
            demand = int(row['Demand'])
            customers.append((latitude, longitude, demand))
    return customers

# Coordinates for depot
depot = (4.4184, 114.0932)

# Vehicles
vehicle_types = [
    ("V1", 25, 1.2),  # Vehicle 1: (type, capacity, cost per km)
    ("V2", 30, 1.5)   # Vehicle 2: (type, capacity, cost per km)
]

# Randomly select number of vehicles, less than 20 for each type
num_v1 = random.randint(1, 19)
num_v2 = random.randint(1, 19)
vehicles = [vehicle_types[0]] * num_v1 + [vehicle_types[1]] * num_v2
random.shuffle(vehicles)

def distance(point1, point2):
    return math.sqrt((point1[0] - point2[0])**2 + (point1[1] - point2[1])**2)

def assign_customers_to_vehicles(vehicles, customers):
    customer_assignments = {i: [] for i in range(len(vehicles))}
    remaining_customers = customers.copy()
    vehicle_indices = list(range(len(vehicles)))

    while remaining_customers:
        random.shuffle(vehicle_indices)
        for i in vehicle_indices:
            if not remaining_customers:
                break
            vehicle = vehicles[i]
            capacity = vehicle[1]
            assigned_load = sum(c[2] for c in customer_assignments[i])
            
            for customer in random.sample(remaining_customers, len(remaining_customers)):
                if assigned_load + customer[2] <= capacity:
                    customer_assignments[i].append(customer)
                    remaining_customers.remove(customer)
                    break

    return customer_assignments

def calculate_cost(vehicles, customer_assignments):
    total_cost = 0
    
    for i, vehicle in enumerate(vehicles):
        if customer_assignments[i]:  # Only calculate cost for vehicles with assigned customers
            route = [depot] + [customer[:2] for customer in customer_assignments[i]] + [depot]
            cost_per_km = vehicle[2]
            
            route_distance = sum(distance(route[j], route[j + 1]) for j in range(len(route) - 1))
            route_cost = route_distance * cost_per_km * 100  # as per provided formula
            total_cost += route_cost
    
    return total_cost

# Path to the CSV file
csv_file_path = 'customer.csv'
customers = read_customers_from_csv(csv_file_path)


# Assign customers to vehicles
customer_assignments = assign_customers_to_vehicles(vehicles, customers)

# Calculate the total trip cost
total_trip_cost = fitness(vehicles, customer_assignments)
print("Total trip cost for all vehicles:", total_trip_cost)


Total trip cost for all vehicles: 279.61759008748703


In [ ]:
#add the genetic algorithm

In [23]:
import csv
import random
import math
import pandas as pd
from copy import deepcopy

# Coordinates for depot
depot = (4.4184, 114.0932)

# Vehicles
vehicle_types = [
    ("V1", 25, 1.2),  # Vehicle 1: (type, capacity, cost per km)
    ("V2", 30, 1.5)   # Vehicle 2: (type, capacity, cost per km)
]

# Read customer data from CSV
def read_customers_from_csv(file_path):
    df = pd.read_csv(file_path)
    customers = []
    for _, row in df.iterrows():
        latitude = float(row['Latitude'])
        longitude = float(row['Longitude'])
        demand = int(row['Demand'])
        customers.append((latitude, longitude, demand))
    return customers

csv_file_path = 'customer.csv'
customers = read_customers_from_csv(csv_file_path)

# Calculate distance between two points
def distance(point1, point2):
    return math.sqrt((point1[0] - point2[0])**2 + (point1[1] - point2[1])**2)

# Assign customers to vehicles
def assign_customers_to_vehicles(vehicles, customers):
    customer_assignments = {i: [] for i in range(len(vehicles))}
    remaining_customers = customers.copy()
    vehicle_indices = list(range(len(vehicles)))

    while remaining_customers:
        random.shuffle(vehicle_indices)
        for i in vehicle_indices:
            if not remaining_customers:
                break
            vehicle = vehicles[i]
            capacity = vehicle[1]
            assigned_load = sum(c[2] for c in customer_assignments[i])
            
            for customer in random.sample(remaining_customers, len(remaining_customers)):
                if assigned_load + customer[2] <= capacity:
                    customer_assignments[i].append(customer)
                    remaining_customers.remove(customer)
                    break

    return customer_assignments

# Calculate fitness (cost) of a chromosome
def fitness(vehicles, customer_assignments):
    total_cost = 0
    
    for i, vehicle in enumerate(vehicles):
        if customer_assignments[i]:  # Only calculate cost for vehicles with assigned customers
            route = [depot] + [customer[:2] for customer in customer_assignments[i]] + [depot]
            cost_per_km = vehicle[2]
            
            route_distance = sum(distance(route[j], route[j + 1]) for j in range(len(route) - 1))
            route_cost = route_distance * cost_per_km * 100  # as per provided formula
            total_cost += route_cost
    
    return total_cost

# Create a random chromosome
def create_chromosome():
    num_v1 = random.randint(1, 19)
    num_v2 = random.randint(1, 19)
    vehicles = [vehicle_types[0]] * num_v1 + [vehicle_types[1]] * num_v2
    random.shuffle(vehicles)
    customer_assignments = assign_customers_to_vehicles(vehicles, customers)
    return (vehicles, customer_assignments)

# Ensure valid crossover
def crossover(parent1, parent2):
    vehicles1, assignments1 = parent1
    vehicles2, assignments2 = parent2
    
    split = random.randint(1, len(customers) - 1)
    
    new_vehicles1 = deepcopy(vehicles1)
    new_vehicles2 = deepcopy(vehicles2)
    
    new_assignments1 = deepcopy(assignments1)
    new_assignments2 = deepcopy(assignments2)
    
    for i in range(len(assignments1)):
        if len(assignments1[i]) > split:
            new_assignments1[i], new_assignments2[i] = new_assignments2[i], new_assignments1[i]
    
    # Ensure all customers are assigned exactly once
    assigned_customers1 = [customer for assignment in new_assignments1.values() for customer in assignment]
    assigned_customers2 = [customer for assignment in new_assignments2.values() for customer in assignment]
    
    missing_customers1 = set(customers) - set(assigned_customers1)
    missing_customers2 = set(customers) - set(assigned_customers2)
    
    for i in range(len(assignments1)):
        for customer in missing_customers1.copy():
            if sum(c[2] for c in new_assignments1[i]) + customer[2] <= vehicles1[i][1]:
                new_assignments1[i].append(customer)
                missing_customers1.remove(customer)
        for customer in missing_customers2.copy():
            if sum(c[2] for c in new_assignments2[i]) + customer[2] <= vehicles2[i][1]:
                new_assignments2[i].append(customer)
                missing_customers2.remove(customer)

    return (new_vehicles1, new_assignments1), (new_vehicles2, new_assignments2)

# Evolution function
def evolve(population):
    graded = [(calculate_cost(chromosome[0], chromosome[1]), chromosome) for chromosome in population]
    graded = [x[1] for x in sorted(graded)]
    retain_length = int(len(graded) * retain_rate)
    
    parents = graded[:retain_length]
    
    desired_length = len(population) - len(parents)
    children = []
    
    while len(children) < desired_length:
        male = random.randint(0, retain_length - 1)
        female = random.randint(0, retain_length - 1)
        
        if male != female:
            male = parents[male]
            female = parents[female]
            
            if random.random() < crossover_rate:
                child1, child2 = crossover(male, female)
                children.append(child1)
                if len(children) < desired_length:
                    children.append(child2)
    
    parents.extend(children)
    return parents

# Genetic algorithm
def genetic_algorithm():
    population = [create_chromosome() for _ in range(population_size)]
    
    for generation in range(generations):
        population = evolve(population)
    
    best_solution = min(population, key=lambda chrom: calculate_cost(chrom[0], chrom[1]))
    best_cost = calculate_cost(best_solution[0], best_solution[1])
    
    print("Best solution:", best_solution)
    print("Best cost:", best_cost)

# Parameters for the genetic algorithm
population_size = 50
generations = 50
retain_rate = 0.2
crossover_rate = 0.8

genetic_algorithm()


Best solution: ([('V2', 30, 1.5), ('V1', 25, 1.2), ('V2', 30, 1.5), ('V2', 30, 1.5), ('V1', 25, 1.2)], {0: [(4.3184, 113.9932, 6), (4.4024, 113.9896, 5)], 1: [(4.3818, 114.2034, 6), (4.3163, 114.0764, 3)], 2: [(4.4142, 114.0127, 8), (4.3976, 114.0049, 8)], 3: [(4.3555, 113.9777, 5), (4.4935, 114.1828, 5)], 4: [(4.4932, 114.1322, 8), (4.4804, 114.0734, 3)]})
Best cost: 220.99670244703006
